# COVID-19 Dashboard Updates
Center for Human Dynamics in the Mobile Age (HDMA) at San Diego State University

Jessica Embury

### MODULES

In [ ]:
#import modules
import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from arcgis.mapping import WebMap
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import json
from copy import deepcopy
import time
import numpy as np
import sys
import webbrowser
from copy import deepcopy
import os 
import requests

### CONFIRM USER VARIABLES BEFORE RUNNING CELLS

In [ ]:
##########################
###SET PATHS IN AND OUT###
##########################
#PATHS IN
#additional needed zip info - community name and population
zips_info_path = './data/zips_info.csv'

#PATHS OUT
#wide df with a column for each date
wide_df_path = './data/covid_accumulated.csv'

#path out for dash updates, feature layer overwrite
path_out= './data/covid_percents_upload.csv'
path_out2 = '../covid_data/percents/covid_percents_{}.csv'

#path to csv used to append new column to feature layer 2 for cumulative cases dashboard
append_csv_path = './data/sd_zip_cumulative_covid_append.csv'

#upload to dropbox
dropbox_cumulative = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/COVID-Data-Share-at-HDMA-Center/SD_Zipcode_COVID_{}.csv'

# 7 day dash layer overwrite
seven_path = 'sandiego_covid_upload_7day.csv'

######################################
###ARCGIS ACCOUNT LOGIN INFORMATION###
######################################

#reference for authentication schemes: https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/

#portal =''
#username=''
#password=''

pro = "pro"

################################
###ARCGIS DETAILS FOR UPDATES###
################################
#URL TO SD COUNTY'S COVID-19 FEATURE SERVICE
sd_dashboard_service = 'https://services1.arcgis.com/1vIhDJwtG5eNmiqX/ArcGIS/rest/services/CovidDashUpdate/FeatureServer'

#feature layer ID to overwrite
feature_layer = "65333d10997d4eb7bf921e11472ae35d"
feature_layer2 = '2a2645b5f569461d916122c3e16d96f3'

#csv to append to feature_layer2
append_csv = '49be034d6b7a406ca291cb44e94e1be1'

#map IDs for symbology update
#heatmaps
hm1 = "3bb63e3ff08243cebf465919191863a0"
hm2 = "2c8a7e1fe2114bfea6317ae4c3268514"

#rates maps
rc1 = "81ff6fce8f7840f8818ae1651db49fc7"
rc2 = "f0d7afc488ae4d048c2413159ab70921"
rc3 = "a24f481d8e954411a40d6b9a18465f86"
rc4 = "a9fe5732f5ac4ac395f66bdd2275d2ec"

#confirmed cases map
cc = "763a114f5f114139af5517ac4c785bd8"

# seven day data and maps
seven_layer = 'f6558646808b4b88ba1d77e984b9f7e8'
seven_map = 'cd2e0028e1f049e7bd268d06c26cfe22'
seven_map_mobile = 'ee46036487c3491caf69209d2a67940c'

#urls for dashboards (to verify)
heatmap_dash = 'https://arcg.is/1br8v9'
rates_dash = 'https://arcg.is/0Gy0mj'
cumulative_dash = 'https://arcg.is/1zXq1m'
cumulative_mobile = 'https://arcg.is/1WLnjG'
seven_dash = 'https://experience.arcgis.com/experience/a630917e020440ba9a598bf1c32b7a74'

### CONNECT TO ARCGIS ACCOUNT
Reference for authentication schemes: https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/

In [ ]:
gis = GIS(pro)

#gis = GIS(portal, username, password)

### COLLECT AND FORMAT COVID-19 DATA FROM SD COUNTY'S FEATURE SERVICE

In [ ]:
#Retrieve the current layer from SDCounty dashboard, convert to a DF, convert to 4326, and transform to a DF by zip codes (using Pivot), resulting in a wide table
db_item = FeatureLayerCollection(sd_dashboard_service)

# convert the layer with counts by zips into a df, in 4326
raw = db_item.layers[0].query(out_sr = 4326, return_geometry=True).sdf

# create the Lat and Lon columns in this df.
raw['Latitude'] = np.nan
raw['Longitude'] = np.nan

for i, row in raw.iterrows():
    temp = raw['SHAPE'][i]
    lat = temp['y']
    lon = temp['x']
    raw['Latitude'][i] = lat
    raw['Longitude'][i] = lon
    
# generate better time stamps
raw['Date'] = raw['UpdateDate']
raw['Date'] = raw['Date'].dt.strftime("%m/%d/%y")
raw = raw.drop_duplicates(['ZipText','Date'])
    
# generate a wide table for the current file, with a column for every date
wide_df = raw.pivot(index='ZipText', columns='Date', values='Case_Count')
wide_df['Zipcode'] = wide_df.index

#GET CURRENT DATE FOR EXPORTED CSVs, CASE RATE AND INCREASE CALCULATIONS
least_recent_date = raw['UpdateDate'].min()
least_recent_date = least_recent_date.strftime("%m/%d/%y")
most_recent_date = raw['UpdateDate'].max()
most_recent_date = most_recent_date.strftime("%m/%d/%y")
print(most_recent_date)

#output wide table as CSV for records
wide_df.to_csv(wide_df_path) #.format(most_recent_date.replace('/', '')))

wide = pd.read_csv(wide_df_path) #.format(most_recent_date.replace('/', '')))

#convert zipcode column to string type
for i, row in wide.iterrows():
    temp = str(wide['Zipcode'][i])
    wide['Zipcode'][i] = temp
    
del wide['ZipText'] 

In [ ]:
#format and export wide df to dropbox
wide_dropbox = wide['Zipcode'].reset_index()
wide_dropbox = wide_dropbox.merge(wide, on = 'Zipcode')
wide_dropbox = wide_dropbox.fillna(0)
wide_dropbox = wide_dropbox.query('Zipcode > 0')
wide_dropbox = wide_dropbox.astype(int)
del wide_dropbox['index']
del wide_dropbox['Unnamed: 1']
wide_dropbox.to_csv(dropbox_cumulative.format(most_recent_date.replace('/','')), index=False)
wide_dropbox

### ASSIGN ZIP COORDS TO DF FOR LATER USE

In [ ]:
#COORDS
#get set of zip code coordinates to add to update df
coords = raw[['ZipText', 'Latitude', 'Longitude']]
coords = coords.rename(columns={'ZipText':'Zipcode'})
coords = coords.drop_duplicates(['Zipcode'])

#convert zipcode column to numeric for merging
coords['Zipcode'] = pd.to_numeric(coords['Zipcode'])

## SAVE SD COUNTY COVID-19 PDF FILES TO DROPBOX

In [ ]:
#specify date (folder name) if different than current feature service date
#most_recent_date = '12/23/20'

#specify directory for new folder
pdf_path = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/{}20'.format(most_recent_date.replace('/', '-'))

#create folder if it doesn't exist
if not os.path.exists(pdf_path):
    os.makedirs(pdf_path)

#base url for pdfs
url = 'https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/'

#pdf names
pdfs = ['COVID-19%20Percentage%20Positive.pdf', 'COVID-19%20Cases%20by%20Date%20of%20Illness%20Onset.pdf', 
        'COVID-19%20Daily%20Update_City%20of%20Residence.pdf', 'COVID-19%20City%20of%20Residence_MAP.pdf', 
        'COVID-19%20Race%20and%20Ethnicity%20Summary.pdf', 'COVID-19%20Summary%20of%20Cases%20by%20Zip%20Code.pdf', 
        'COVID-19%20Hospitalizations%20by%20Date%20Admitted.pdf', 'COVID-19%20Hospitalizations%20Summary_ALL.pdf', 
        'COVID-19%20Deaths%20by%20Date%20of%20Death.pdf', 'COVID-19%20Deaths%20by%20Demographics.pdf', 
        'COVID-19_Daily_Status_Update.pdf', 'COVID-19%20Watch.pdf', 'Summary_County_of_San_Diego_Supported_Tests_by_Race_Ethnicity.pdf',
        'Summary_of_All_Tests_Reported_by_Race_Ethnicity.pdf', 'Summary_of_All_Tests_Reported_by_Zip_Code_of_Residence.pdf', 
        'Summary_Tests_Among_San_Diego_County_Residents_by_Race_Ethnicity.pdf']

#for each pdf in list, get and save
for i in range(len(pdfs)):
    response = requests.get(url + pdfs[i], stream=True)

    with open(pdf_path + '/' + pdfs[i].replace('%20', ' '), 'wb') as f:
        f.write(response.content)

#pdf with a different url
response = requests.get(url + '/covid19/MediaBriefingSlides/mediaBriefingSlides.pdf', stream=True)
with open(pdf_path + '/' + 'mediaBriefingSlides.pdf', 'wb') as f:
        f.write(response.content)

### MERGE WIDE DF WITH COORDINATES DF AND COMMUNITY/POPULATION CSV DATA
### SUBSET DF WITH ONLY COLUMNS FOR HDMA RATES/PERCENTS FEATURE LAYER

In [ ]:
#import additional needed zip info
zips_info = pd.read_csv(zips_info_path)

for i, row in zips_info.iterrows():
    temp = str(zips_info['Zipcode'][i])
    zips_info['Zipcode'][i] = temp

#merge with coords to add lat and lon
zips_info = zips_info.merge(coords, on="Zipcode")

#merge wide df with coords and extra zip code info
wide = wide.merge(zips_info, on="Zipcode")

#create new df for use feature layer overwrite
cols = ['Zipcode', 'Community', 'Latitude', 'Longitude', '2018_population']
df = wide[cols]

df['Date'] = most_recent_date

df['Confirmed Cases'] = wide[most_recent_date]
df['Confirmed Cases'] = df['Confirmed Cases'].fillna(0)
df['Confirmed Cases'] = df['Confirmed Cases'].astype(int)

df['Rate Per 100K'] = (df['Confirmed Cases']/df['2018_population']*100000).round(2)

df = df.fillna(0)

### TEMPORARY DF TO CALCULATE CASE INCREASE AND RATES OF CHANGE, MERGE TO MAIN DF

In [ ]:
#CREATE DF
date_df = wide[['Zipcode', wide.columns[-13], wide.columns[-12], wide.columns[-11], wide.columns[-10], wide.columns[-9], wide.columns[-8], wide.columns[-7], wide.columns[-6]]]
date_df = date_df.fillna(0)
#date_df.head()

#CREATE COLUMN FOR CASE INCREASES AND CALCULATE
date_df['Daily Increased'] = 0

for i, row in date_df.iterrows():
    day1 = int(date_df.iloc[:,-2][i])
    day2 = int(date_df.iloc[:,-3][i])
    date_df['Daily Increased'][i] = (day1-day2)

#CREATE COLUMN FOR DAILY CHANGE RATE AND CALCULATE
date_df['Daily Change Rate*1000'] = 0.0

for i, row in date_df.iterrows():
    day1 = int(date_df.iloc[:,-3][i])
    day2 = int(date_df.iloc[:,-4][i])
    if(day2 != 0):
        date_df['Daily Change Rate*1000'][i] = round(((day1 - day2)/day2)*1000, 2)

#CREATE COLUMN FOR 7 DAY ROLLING RATE OF CHANGE AND CALCULATE
date_df['7 Days Rolling Change*1000'] = 0.0

rolling_rate_day_list_cols = [[-4,-5],[-5,-6],[-6,-7],[-7,-8],[-8,-9],[-9,-10],[-10,-11]]

for i, row in date_df.iterrows():
    all_days=0
    
    for x in range(len(rolling_rate_day_list_cols)):
        day1 = int(date_df.iloc[:,rolling_rate_day_list_cols[x][0]][i])
        day2 = int(date_df.iloc[:,rolling_rate_day_list_cols[x][1]][i])

        if(day2 != 0):
            one_day = ((day1 - day2)/day2)*1000
            all_days = all_days + one_day
           
    date_df['7 Days Rolling Change*1000'][i] = round(all_days/len(rolling_rate_day_list_cols), 2)
    

# create column for 7 day case increase and calculate
date_df['7 Day Case Increase'] = 0

for i, row in date_df.iterrows(): 
    seven_change = date_df.iloc[:,8][i] - date_df.iloc[:,1][i]
    date_df['7 Day Case Increase'][i] = int(seven_change)

#MERGE NEW COLUMNS WITH MAIN DF
date_df_subset = date_df[['Zipcode', 'Daily Increased', 'Daily Change Rate*1000', '7 Days Rolling Change*1000', '7 Day Case Increase']]

df =df.merge(date_df_subset, on='Zipcode')
#print(len(df))
#df.head()

#SUBSET DATA TO POPULATION >= 5000 AND CONFIRMED CASES >= 10
df = df[df['2018_population'] >= 5000]
df = df[df['Confirmed Cases'] >= 10]
#print(len(df))
#df.head()

### CALCULATE NEW/CUMULATIVE CASE PERCENTS

In [ ]:
#get cumulative case and daily increase totals for county
sum_confirmed = 0
sum_daily = 0

for i, row in df.iterrows():
    sum_confirmed = sum_confirmed + df['Confirmed Cases'][i]
    sum_daily = sum_daily + df['Daily Increased'][i]
    
#add new columns and calculate percent of total and percent of daily new for each zip code
df['percent_total'] = (df['Confirmed Cases']/sum_confirmed)*100
df['percent_daily'] = (df['Daily Increased']/sum_daily)*100

#round columns to 2 decimal places
df['percent_total'] = df['percent_total'].round(2)
df['percent_daily'] = df['percent_daily'].round(2)

#no decimals for zipcode column
df['Zipcode'] = df['Zipcode'].apply(np.int64)

#df.head()

### OUTPUT CSV FOR RECORDS

In [ ]:
#save as csv files
df.to_csv(path_out, index = False) 
df.to_csv(path_out2.format(most_recent_date.replace('/','')), index = False)
df.head()

# UPDATE MAP SYMBOLOGY FOR RATES AND HEATMAP DASHBOARDS

Reference: https://community.esri.com/groups/arcgis-python-api/blog/2019/04/09/updating-layer-symbology-with-the-arcgis-api-for-python

### FEATURE LAYER OVERWRITE

In [ ]:
#get feature layer containing updated data for maps associated with the COVID-19 dashboards
layer = gis.content.get(feature_layer)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out)

### FUNCTIONS

In [ ]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''
    
    m = gis.content.get(map_id)

    data = m.get_data()
    
    #Include the below line for prettified JSON
    #print(json.dumps(data, indent=4, sort_keys=True))

    print(m)
    
    return data
    
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)
    
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}

    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)
    
    return update

### FIND MAX/MIN VALUES FOR MAP SYMBOLOGY CHANGES

In [ ]:
max_confirmed = df['Confirmed Cases'].max()

max_increased = df['Daily Increased'].max()
max_decreased = 0 - max_increased

min_rate100k = df['Rate Per 100K'].min()
max_rate100k = df['Rate Per 100K'].max()

min_7dayrate = df['7 Days Rolling Change*1000'].min()
max_7dayrate = df['7 Days Rolling Change*1000'].max()

max_7day = df['7 Day Case Increase'].max()

### RATES OF CHANGE DASHBOARD

In [ ]:
#rates map1: confirmed cases

#get map data
rc1_data = get_map(rc1)

#adjust symbology for graduated points to reflect new max/min
#MAX CONFIRMED
rc1_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_confirmed.item()
rc1_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_confirmed.item()

#MAX CONFIRMED
rc1_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_confirmed.item()
rc1_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_confirmed.item()

#update map to save changes
rc1_update = update_map(rc1, rc1_data)
rc1_update

In [ ]:
#rates map2: daily case increases

#get map data
rc2_data = get_map(rc2)

#adjust symbology for graduated points to reflect new max/min
#MAX DAILY INCREASE
rc2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_increased.item()
rc2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_increased.item()

#MAX DAILY INCREASE
rc2_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_increased.item()
rc2_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_increased.item()

#update map to save changes
rc2_update = update_map(rc2, rc2_data)
rc2_update

In [ ]:
#rates map3: cases per 100k residents

#get map data
rc3_data = get_map(rc3)

#adjust symbology for graduated points to reflect new max/min
#MAX RATE PER 100K
rc3_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_rate100k.item()
rc3_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate100k.item()

#MIN RATE PER 100K
rc3_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['minSliderValue'] = min_rate100k.item()
rc3_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['minDataValue'] = min_rate100k.item()

#MAX RATE PER 100K
rc3_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_rate100k.item()
rc3_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate100k.item()

#MIN RATE PER 100K
rc3_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['minSliderValue'] = min_rate100k.item()
rc3_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['minDataValue'] = min_rate100k.item()

#update map to save changes
rc3_update = update_map(rc3, rc3_data)
rc3_update

In [ ]:
#rates map4: 7 day rolling rate of change

#get map data
rc4_data = get_map(rc4)

#adjust symbology for graduated points to reflect new max/min
#MAX 7 DAY RATE
rc4_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7dayrate.item()
rc4_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7dayrate.item()

#MIN 7 DAY RATE
rc4_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['minSliderValue'] = min_7dayrate.item()
rc4_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['minDataValue'] = min_7dayrate.item()

#MAX 7 DAY RATE
rc4_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7dayrate.item()
rc4_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7dayrate.item()

#MIN 7 DAY RATE
rc4_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['minSliderValue'] = min_7dayrate.item()
rc4_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['minDataValue'] = min_7dayrate.item()

#update map to save changes
rc4_update = update_map(rc4, rc4_data)
rc4_update

### HEATMAPS DASHBOARD

In [ ]:
#heatmap1: daily case increases

#get map data
hm1_data = get_map(hm1)

#adjust symbology for graduated points to reflect new max/min
#DAILY INCREASED POINTS
hm1_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_increased.item()
hm1_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_increased.item()

#DAILY DECREASED POINTS
hm1_data['operationalLayers'][8]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['minSliderValue'] = max_decreased.item()
hm1_data['operationalLayers'][8]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['minDataValue'] = max_decreased.item()

#update map to save changes
hm1_update = update_map(hm1, hm1_data)
hm1_update

In [ ]:
#heatmap2: 7 day rolling rate of change

#get map data
hm2_data = get_map(hm2)

#adjust symbology for graduated points to reflect new max/min
#MAX RATE
hm2_data['operationalLayers'][7]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7dayrate.item()
hm2_data['operationalLayers'][7]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7dayrate.item()

#MIN RATE
hm2_data['operationalLayers'][7]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['minSliderValue'] = min_7dayrate.item()
hm2_data['operationalLayers'][7]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['minDataValue'] = min_7dayrate.item()

#update map to save changes
hm2_update = update_map(hm2, hm2_data)
hm2_update

# UPDATE CUMULATIVE CASE DASHBOARD

### APPEND DATA TO CUMULATIVE CASES FEATURE LAYER
Reference: https://developers.arcgis.com/python/guide/appending-features/

In [ ]:
#get feature layer containing updated data for maps associated with the CUMULATIVE COVID-19 dashboard
layer2 = gis.content.get(feature_layer2)
layer2

In [ ]:
#List all the current fields in the layer so you can use one as a field template.
cum_covid_lyr = layer2.layers[0]
cum_covid_lyr

In [ ]:
#reformat date for use in appending: append_source = column name in csv, append_field = column name in feature layer, append_alias = column alias
split = most_recent_date.split('/')

append_source = ''

for i in range(len(split)):
    if(split[i][0] is '0'):
        split[i] = split[i][1:]
    if(i == 0):
        append_source += split[i]
    elif(i == (len(split)-1)): 
        append_source += '_' + split[i]
    else:
        append_source += '_' + split[i]

append_field = 'F' + append_source + '20'
append_alias = append_source.replace('_','/') + '20'

In [ ]:
#Create a dictionary from a deep copy of a field in the feature layer, and update the values of this dictionary to reflect a new field.
new_field = dict(deepcopy(cum_covid_lyr.properties.fields[5]))
new_field['name'] = append_field
new_field['alias'] = append_alias
new_field['length'] = "10"
print(new_field)

#Update feature layer definition with the new field using the add_to_definition() method.
field_list = [new_field]
cum_covid_lyr.manager.add_to_definition({"fields":field_list})

In [ ]:
#only need to add index to 'Zipcode' once - cell kept for reference

#Add a unique index to the new attribute field, needed to append
#flds = [f.fields.lower() for f in cum_covid_lyr.properties.indexes if f.isUnique]

#for fld in cum_covid_lyr.properties.fields:
#    if fld.name.lower() in flds:
#        print(f"{fld.name:30}{fld.type:25}isUnique")
#    else:
#        print(f"{fld.name:30}{fld.type:25}")

#Create a copy of one index, then edit it to reflect values for a new index. Then add that to the layer definition.
#name_idx = dict(deepcopy(cum_covid_lyr.properties['indexes'][0]))
#name_idx['name'] = 'Zipcode'
#name_idx['fields'] = 'Zipcode'
#name_idx['isUnique'] = True
#name_idx['description'] = 'index_name'
#name_idx

#index_list = [name_idx]
#cum_covid_lyr.manager.add_to_definition({"indexes":index_list})

#Verify the index was added
#layer2 = gis.content.get(feature_layer2)
#layer2

#flds = [f.fields.lower() for f in cum_covid_lyr.properties.indexes if f.isUnique]

#for fld in cum_covid_lyr.properties.fields:
#    if fld.name.lower() in flds:
#        print(f"{fld.name:30}{fld.type:25}isUnique")
#    else:
#        print(f"{fld.name:30}{fld.type:25}")      

In [ ]:
#update csv item to contain new date column for addition to the feature layer
append_df2 = wide_df[['Zipcode', '{}'.format(most_recent_date)]]
append_df2 = append_df2.rename(columns = {'{}'.format(most_recent_date): '{}'.format(append_source)})

append_df2 = append_df2[append_df2.index.notnull()]
append_df2 = append_df2.fillna(0)
append_df2['{}'.format(append_source)] = append_df2['{}'.format(append_source)].astype(int)
append_df2.to_csv(append_csv_path)


In [ ]:
append_csv_item = gis.content.get(append_csv)
#append_csv_item
append_csv_item.update({}, append_csv_path)

#get *append_csv_info* when appending a new column for source_info
append_csv_info = gis.content.analyze(item=append_csv, file_type='csv', location_type='none')
#append_csv_info

In [ ]:
#append new date column to feature layer from csv item
cum_covid_lyr.append(item_id= append_csv,
                      upload_format = 'csv',
                      field_mappings = [{"name":"{}".format(append_field), "source":"{}".format(append_source)},
                                        {"name":"Zipcode", "source":"Zipcode"}],
                      source_info = append_csv_info['publishParameters'],
                      update_geometry=False,
                      append_fields=["{}".format(append_field), "Zipcode"],
                      skip_inserts=True,
                      upsert_matching_field="Zipcode")

### MODIFY CONFIRMED CASES WEB MAP SYMBOLOGY

In [ ]:
#confirmed cases map

#get map data
cc_data = get_map(cc)

#set symbol to new date field, adjust max symbology
cc_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['field'] = append_field
cc_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['field'] = append_field
cc_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_confirmed.item()
cc_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_confirmed.item()

#set labeling to new date field
cc_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['labelingInfo'][0]['labelExpressionInfo']['expression'] = '$feature["{}"]'.format(append_field)
cc_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['labelingInfo'][0]['labelExpressionInfo']['value'] = ('{' + append_field + '}')
cc_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['labelingInfo'][0]['fieldInfos'][0]['fieldName'] = append_field

#set filter to new date
cc_data['operationalLayers'][2]['layerDefinition']['definitionExpression'] = ('{} > 0'.format(append_field))

#adjust last date in popup
new_date = cc_data['operationalLayers'][2]['popupInfo']['fieldInfos'][-1].copy()
new_date['fieldName'] = append_field
new_date['label'] = append_alias
cc_data['operationalLayers'][2]['popupInfo']['fieldInfos'][-1]['visible'] = False
cc_data['operationalLayers'][2]['popupInfo']['fieldInfos'].append(new_date)
#cc_data['operationalLayers'][2]['popupInfo']['fieldInfos']

#add new date to popup chart
popup_chart=cc_data['operationalLayers'][2]['popupInfo']['mediaInfos'][0]['value']['fields']
popup_chart.append(append_field)
cc_data['operationalLayers'][2]['popupInfo']['mediaInfos'][0]['value']['fields'] = popup_chart

#update map to save changes
cc_update = update_map(cc, cc_data)
cc_update

# UPDATE 7 DAY DASHBOARD

### OVERWRITE

In [ ]:
#get feature layer containing updated data for maps associated with the COVID-19 dashboards
layer = gis.content.get(seven_layer)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out)

### MAP SYMBOLOGY

In [ ]:
#seven day map

#get map data
sm_data = get_map(seven_map)

#adjust symbology for graduated points to reflect new max/min
#MAX DAILY INCREASE
sm_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7day.item()
sm_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7day.item()

#update map to save changes
sm_update = update_map(seven_map, sm_data)
sm_update

#seven day map mobile

#get map data
smm_data = get_map(seven_map_mobile)

#adjust symbology for graduated points to reflect new max/min
#MAX DAILY INCREASE
smm_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7day.item()
smm_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7day.item()

#update map to save changes
smm_update = update_map(seven_map_mobile, smm_data)
smm_update

# VIEW UPDATED DASHBOARDS

In [ ]:
#Open heatmaps dashboard in browser
webbrowser.open(heatmap_dash, new=2)

#Open rates of change dashboard in browser
webbrowser.open(rates_dash, new=2)

#open cumulative/growth chart map
webbrowser.open(cumulative_dash, new=2)

#open seven day cases dash
webbrowser.open(seven_dash, new=2)

#County-wide information snapshot
print("Date: {}, Number of Zip Codes: {}, Total Cases: {}, Daily Increase: {}".format(most_recent_date, len(df), sum_confirmed, sum_daily))
#df.head()